In [ ]:
! pip install pycountry-convert --user
! pip install geopy --user
! pip install folium --user

In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import gzip
import math
from subprocess import check_output
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
%matplotlib inline

In [ ]:
production_df = pd.read_excel('../data/tmdb_production_data.xlsx', index_col=0)
movie_df = pd.read_excel('../data/tmdb_movie_data.xlsx', index_col=0)
genre_df = pd.read_excel('../data/movie_genres_data.xlsx', index_col=0)

In [ ]:
production_movie_df = pd.merge(production_df, movie_df, left_on='movie_title', right_on='movie_title', how='inner')
display(production_movie_df)

In [ ]:
countries_movie_count = pd.DataFrame(production_movie_df.groupby(['origin_country'])['origin_country'].count())
countries_movie_count.rename(columns = {'origin_country':'movie_count'}, inplace = True)
countries_movie_count = countries_movie_count.reset_index()
display(countries_movie_count)

In [ ]:
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)

In [ ]:
x = 0
codes = []
while x < len(countries_movie_count['origin_country']):
    codes.append(get_continent(countries_movie_count['origin_country'][x]))
    x=x+1
countries_movie_count['codes'] = codes

In [ ]:
country = []
continent = []
x=0
while x < len(codes):
    country.append(codes[x][0])
    continent.append(codes[x][1])
    x=x+1
countries_movie_count['country'] = country
countries_movie_count['continent'] = continent

In [ ]:
countries_movie_count

In [ ]:
geolocator = Nominatim(user_agent='my_application')
def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return np.nan

In [ ]:
x = 0
cords = []
while x < len(countries_movie_count['country']):
    cords.append(geolocate(countries_movie_count['country'][x]))
    x=x+1
countries_movie_count['cords'] = cords

In [ ]:
latitude = []
longitude = []
x=0
while x < len(cords):
    try:
        latitude.append(cords[x][0])
    except TypeError:
        latitude.append('nan')
    try:
        longitude.append(cords[x][1])
    except TypeError:
        longitude.append('nan')
    x=x+1
countries_movie_count['latitude'] = latitude
countries_movie_count['longitude'] = longitude

In [ ]:
countries_movie_count.dropna(subset=['cords'], inplace=True)

In [ ]:
countries_movie_count

In [ ]:
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(countries_movie_count)):
        lat = countries_movie_count.iloc[i]['latitude']
        long = countries_movie_count.iloc[i]['longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Number of movies : {}<br>"""
        popup_text = popup_text.format(countries_movie_count.iloc[i]['country'],
                                   countries_movie_count.iloc[i]['movie_count']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map

In [ ]:
continent_counts = countries_movie_count[['movie_count', 'continent']]
continent_counts = continent_counts.groupby('continent').sum()
continent_counts = continent_counts.reset_index()
continent_counts = continent_counts.replace(to_replace=['AF', 'AS', 'EU', 'NA', 'OC', 'SA'], value=['Africa', 'Asia', 'Europe', 'North America', ''])
display(continent_counts)